## Board Games

In [1]:
import pandas as pd
import numpy as np
import math 

def calculate_score(row, name='M'):
    score_list = str(row).split("+")
    for score in score_list:
        score_alpha = ''.join(x for x in score if x.isalpha())
        if name == score_alpha:
            score_digit = ''.join(c for c in score if c.isdigit())
            return score_digit
    return None

def calculate_individual_score(df):
    """ Calculate the scores for each individual and returns a new column for each person
    """
    
    people = {"Ilse": "I", "Maarten": "M", "Edith": "E", "Guy": "G", "Mam": "MA",
             "Pap": "PA", "Wouter": "WO", "Evi": "EV", "Iris": "IR", "Daan": "DA", 
             "Christopher": "C"}
    for key, item in people.items():
        df['{}_points'.format(key)] = df.apply(lambda row: calculate_score(row.Scores, item), 1)  
        
    return df

def calculate_won(row, person):   
    """ Calculates if a person has won that game based on whether its name appears
    in the Winner column
    
    """
    if (type(row) == str):
        if person == row:
            return 1
        else:
            return 0
    else:
        if (math.isnan(row)) & (type(person)!=str):
            return 1
        else:
            return 0
        
def prepare_ilse_vs_maarten(df):
    ilse_vs_maarten = df.loc[(df.Players=="I+M"), :]
    ilse_vs_maarten = ilse_vs_maarten[['Date', 'Players', 'Game', 'Scores', 'Winner', 'Version',
                                      'Ilse_points', 'Maarten_points']].copy()
    ilse_vs_maarten.Ilse_points = ilse_vs_maarten.Ilse_points.astype(float)
    ilse_vs_maarten.Maarten_points = ilse_vs_maarten.Maarten_points.astype(float)
    
    for column, person in zip(['Ilse_won', 'Maarten_won', 'Draw', 'Lost'],
                              ['I', 'M', 'I+M', None]):
            ilse_vs_maarten[column] = ilse_vs_maarten.apply(lambda row: calculate_won(row.Winner, person), 1)

    return ilse_vs_maarten
    

In [2]:
df = pd.read_excel("matches.xlsx")
df = calculate_individual_score(df)

In [4]:
df.head()

,Date,Players,Game,Scores,Winner,Version,Ilse_points,Maarten_points,Edith_points,Guy_points,Mam_points,Pap_points,Wouter_points,Evi_points,Iris_points,Daan_points,Christopher_points
0,2018-11-18,I+M,Qwixx,I77+M77,I+M,Normal,77,77,None,None,None,None,None,None,None,None,None
1,2018-11-18,I+M,Qwixx,I104+M65,I,Normal,104,65,None,None,None,None,None,None,None,None,None
2,2018-11-18,I+M,Qwixx,I96+M62,I,Normal,96,62,None,None,None,None,None,None,None,None,None
3,2018-11-18,I+M,Mind,LVL10,NaN,Normal,None,None,None,None,None,None,None,None,None,None,None
4,2018-11-18,I+M,Mind,LVL4,NaN,Normal,None,None,None,None,None,None,None,None,None,None,None


## Ilse vs. Maarten

In [12]:
ilse_vs_maarten = prepare_ilse_vs_maarten(df)
ilse_vs_maarten.head(5)

,Date,Players,Game,Scores,Winner,Version,Ilse_points,Maarten_points,Ilse_won,Maarten_won,Draw,Lost
0,2018-11-18,I+M,Qwixx,I77+M77,I+M,Normal,77.0,77.0,0,0,1,0
1,2018-11-18,I+M,Qwixx,I104+M65,I,Normal,104.0,65.0,1,0,0,0
2,2018-11-18,I+M,Qwixx,I96+M62,I,Normal,96.0,62.0,1,0,0,0
3,2018-11-18,I+M,Mind,LVL10,NaN,Normal,NaN,NaN,0,0,0,1
4,2018-11-18,I+M,Mind,LVL4,NaN,Normal,NaN,NaN,0,0,0,1


In [48]:
matches = (ilse_vs_maarten.groupby(['Game', 'Players', 'Version'])
                          .agg({"Ilse_points":['mean', 'median', 'std'],
                                "Maarten_points":['mean', 'median', 'std'],
                                "Ilse_won":['sum'],
                                "Maarten_won":['sum'],
                                "Draw":['sum'],
                                "Lost":['sum']})
                          .reset_index())
matches.columns = ['Game', 'Players', 'Version', 'Lost', 'I_mean', 'I_median', 'I_std', 
                     'I_won', 'Draw', "M_"
                    'M_mean', 'M_median', 'M_std', 'M_won']
matches['Total'] = matches.apply(lambda row: row.Lost+row.M_won+row.I_won+row.Draw, 1)

In [50]:
matches['Full_game'] = matches.apply(lambda row: row.Game + row.Version if row.Version != 'Normal' 
                                     else row.Game, 1)

In [54]:
file = [{"Name": "Ilse"}]
for row in matches.iterrows():
    file[0][row[1].Full_game] = row[1]['I_won']

In [55]:
file

[{'7 Wonders Duel': 1,
  'Claim': 3,
  'Claim 2': 1,
  'Fox in the Forest': 2,
  'Jaipur': 6,
  'KeyForge': 4,
  'Kingdomino 5x5': 5,
  'Kingdomino 7x7': 1,
  'Mind': 0,
  'Name': 'Ilse',
  'Port Royal': 2,
  'Qwixx': 19,
  'QwixxColors': 2,
  'QwixxRandom': 2,
  'Raptor': 2,
  'Regenwormen': 4,
  'Rozenkoning': 1,
  'Santorini': 0,
  'Schotten Totten': 7,
  'Splendor': 0,
  'Sushi Go!': 2,
  'UnlockDe Formule': 0,
  'UnlockMuis en de Worst': 0,
  'Welcome to…': 0,
  "What's up": 1}]